In [1]:
#!pip install scikit-learn --force
#!pip install catboost

In [5]:
import sys
sys.path.append('../..')
import pandas as pd

pd.set_option('display.max.columns', 300)


# from core.calculator.deposits import DepositsCalculationType
# from core.calculator.deposits import DepositIterativeCalculator

from core.definitions import *
#from core.project_update import load_portfolio

#from core.models import DepositModels
from warnings import filterwarnings
filterwarnings('ignore')

## Корректируем таблицу по НС

In [6]:
# для создания табличек

from core.models.utils import run_spark_session
spark = run_spark_session('create_table_tnd_v2')

#spark = None #если без обучения

In [7]:
query2 = """

with agg_t
as
(
with df 
as

(
with dep 
as
(
SELECT * 
FROM prod_cdm.cd_deposit
where deposit_type_cd='SA'
and passive_acct_type_cd='CUR_ACCT'
AND open_dt>'2010-01-01'
AND deleted_flg='0'
AND substring(account_num, 1, 5) in ('20309', '20310',
        '30601', '30606', '30220', '30223', '30227', 
        '40803', '40810', '40813', '40817', '40820', '40823', '42301', '42309', '42601', '42609', '40901', 
        '40902', '40903', '40905', '40906', '40909', '40910', '40911', '40912', '40913', '40914', '47418')
        
and effective_from_dttm <= '2023-07-31'
and effective_to_dttm >= '2023-07-01'

)


select 
report_date, 
agreement_rk, 
currency_iso_cd, 
open_dt, 
close_dt, 
product_bank_rk, 
product_rk, 
deposit_rate,
interest_period_desc,
account_num, 
currency_rk 

from dep
left join
(
SELECT * 
FROM dadm_alm_sbx.t_enhanced_calendar
where report_date >= '2023-07-01'
and report_date <= '2023-07-31'
) cal
on (cal.report_date between dep.effective_from_dttm and dep.effective_to_dttm)
and cal.report_date >= dep.open_dt 
and cal.report_date <= dep.close_dt 


)

select 
report_date, 
df.agreement_rk, 
currency_iso_cd, 
open_dt, 
close_dt, 
product_bank_rk, 
product_rk, 
deposit_rate,
interest_period_desc,
account_num, 
df.currency_rk,
balance_rur_amt, 
balance_amt

from df
left join
(select 
agreement_rk,
currency_rk,
balance_rur_amt,
balance_amt,
effective_from_dttm,
effective_to_dttm
from prod_cdm.fct_deposit_balance
where balance_amt>1
and deleted_flg='0') end_bal
ON end_bal.agreement_rk = df.agreement_rk
AND end_bal.currency_rk = df.currency_rk
AND (df.report_date BETWEEN end_bal.effective_from_dttm AND end_bal.effective_to_dttm)
and report_date >='2023-06-01'
and report_date <='2023-08-01'
)


select sum(min_balance) as value from
(
select min(balance_rur_amt) as min_balance
from agg_t
group by agreement_rk) t1



"""

In [8]:
df = spark.sql(query2)

[Stage 0:==============================================>       (190 + 33) / 223]23/10/25 17:20:38 ERROR cluster.YarnScheduler: Lost executor 9 on p0drp0-db5031xp.region.vtb.ru: Unable to create executor due to Address already in use: Service 'org.apache.spark.network.netty.NettyBlockTransferService' failed after 5 retries (starting from 62095)! Consider explicitly setting the appropriate port for the service 'org.apache.spark.network.netty.NettyBlockTransferService' (for example spark.ui.port for SparkUI) to an available port or increasing spark.port.maxRetries.
23/10/25 17:20:38 ERROR cluster.YarnScheduler: Lost executor 12 on p0drp0-db5031xp.region.vtb.ru: Unable to create executor due to Address already in use: Service 'org.apache.spark.network.netty.NettyBlockTransferService' failed after 5 retries (starting from 62095)! Consider explicitly setting the appropriate port for the service 'org.apache.spark.network.netty.NettyBlockTransferService' (for example spark.ui.port for SparkUI)

In [9]:
df.write.mode('overwrite').saveAsTable('dadm_alm_sbx.tmp_jun_sa_daily_value2')

[Stage 7:>                                                          (0 + 1) / 1]23/10/25 18:08:36 ERROR cluster.YarnScheduler: Lost executor 45 on p0drp0-db5042xp.region.vtb.ru: Unable to create executor due to Address already in use: Service 'org.apache.spark.network.netty.NettyBlockTransferService' failed after 5 retries (starting from 62095)! Consider explicitly setting the appropriate port for the service 'org.apache.spark.network.netty.NettyBlockTransferService' (for example spark.ui.port for SparkUI) to an available port or increasing spark.port.maxRetries.
23/10/25 18:08:36 ERROR cluster.YarnScheduler: Lost executor 35 on p0drp0-db5042xp.region.vtb.ru: Unable to create executor due to Address already in use: Service 'org.apache.spark.network.netty.NettyBlockTransferService' failed after 5 retries (starting from 62095)! Consider explicitly setting the appropriate port for the service 'org.apache.spark.network.netty.NettyBlockTransferService' (for example spark.ui.port for SparkUI

In [10]:
tmp = spark.table('dadm_alm_sbx.tmp_jun_sa_daily_value2')

In [11]:
tmp.show()

[Stage 8:>                                                          (0 + 1) / 1]

+-------------------+
|              value|
+-------------------+
|1162128190575.78000|
+-------------------+



In [ ]:
1162128190575

In [ ]:
1066878751900.35000/ 10**9

## Калькулятор маржи

Внутренняя стоимость средств на накопительных счетах (ВССНС) ФЛ в рублях  
устанавливается в виде фиксированной ставки как сумма значения Ключевой  
ставки Банка России и надбавки, равной среднекалендарному значению  
трансфертной ставки в рублях на базе КС на срок 1 год за предшествующие дате  
расчета 1095 дней (с учетом выходных и праздничных дней).  